In [38]:
import numpy as np
import matplotlib.pyplot as plt
import inspect
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import os

In [39]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

#print(sys.path)


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [40]:
x_train, y_train,x_test_original, y_test_original = ReadTS('OSUleaf')
y_train = y_train - 1
y_test_original = y_test_original - 1
#print(x_train.shape)
#print(y_train)

In [41]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):

    original_shape = TS.shape
    
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    scaler = StandardScaler()
    
    TS_normalized = scaler.fit_transform(TS_2d)
    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train[1])

In [42]:
model = make_CNN_model(x_train.shape[1:],num_classes = 6)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=500, batch_size=32)

Epoch 1/500
5/5 [==============================] - 0s 47ms/step - loss: 1.7778 - sparse_categorical_accuracy: 0.2250 - val_loss: 1.7956 - val_sparse_categorical_accuracy: 0.1500
Epoch 2/500
5/5 [==============================] - 0s 30ms/step - loss: 1.5838 - sparse_categorical_accuracy: 0.4500 - val_loss: 1.7944 - val_sparse_categorical_accuracy: 0.1500
Epoch 3/500
5/5 [==============================] - 0s 30ms/step - loss: 1.4871 - sparse_categorical_accuracy: 0.4688 - val_loss: 1.7941 - val_sparse_categorical_accuracy: 0.1500
Epoch 4/500
5/5 [==============================] - 0s 28ms/step - loss: 1.4009 - sparse_categorical_accuracy: 0.4750 - val_loss: 1.7942 - val_sparse_categorical_accuracy: 0.1500
Epoch 5/500
5/5 [==============================] - 0s 29ms/step - loss: 1.3586 - sparse_categorical_accuracy: 0.4500 - val_loss: 1.7971 - val_sparse_categorical_accuracy: 0.1500
Epoch 6/500
5/5 [==============================] - 0s 26ms/step - loss: 1.3114 - sparse_categorical_accuracy: 

In [43]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

8/8 [==============================] - 0s 4ms/step - loss: 5.8084 - sparse_categorical_accuracy: 0.5124
Test accuracy 0.5123966932296753
Test loss 5.808360576629639


In [44]:
def l2norm(perturb):
    difference = perturb - x_test_original
    
    squared_difference = np.square(difference)
    
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    #print(average_l2_norm)

    return average_l2_norm


def perturb_predict(strategy,index_0,index_1):
    if strategy =='RBP1fast':
        bg = multisamplebackgroundIdentification(perturb_test_set)
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1,bg = bg) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)
    else:
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)

    accuracy = accuracy_score(y_test_original, predictions_pert)
    precision = precision_score(y_test_original, predictions_pert, average='weighted')  
    recall = recall_score(y_test_original, predictions_pert, average='weighted')
    f1 = f1_score(y_test_original, predictions_pert, average='weighted')
    l2 = l2norm(x_test_pert)

    # print(strategy)
    # print(accuracy)
    # print(precision)
    # print(recall)
    # print(f1)

    return accuracy,precision,recall,f1,l2

In [60]:
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 80
index_1 = 200

strategies = ['RBP', 'RBP1fast', 'zero', 'noise', 'blur']

results = []
l2_results = []

for strategy in strategies:
    accuracy, precision, recall, f1, l2 = perturb_predict(strategy, index_0, index_1)
    results.append([strategy, accuracy, precision, recall, f1])
    l2_results.append(l2)

columns = ['Strategy', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
df_results = pd.DataFrame(results, columns=columns)
l2_columns = ['L2-Norm']
l2_results = pd.DataFrame(l2_results, columns=l2_columns)

predictions_original = model.predict(x_test_original)
predictions_original = np.argmax(predictions_original,axis=1)
accuracy = accuracy_score(y_test_original, predictions_original)
precision = precision_score(y_test_original, predictions_original, average='weighted')  
recall = recall_score(y_test_original, predictions_original, average='weighted')
f1 = f1_score(y_test_original, predictions_original, average='weighted')
original_data = ['original', accuracy, precision, recall, f1]
df_results.loc[-1] = original_data  
df_results.index = df_results.index + 1  
df_results = df_results.sort_index()  

df_results.loc[2, 'Strategy'] = 'MRBP'

current_directory = os.getcwd()
result_folder = os.path.join(current_directory, 'result_data')

csv_path = os.path.join(result_folder, 'OSUleaf.csv')
df_results.to_csv(csv_path, index=False)

csv_path = os.path.join(result_folder, 'OSUleafl2.csv')
l2_results.to_csv(csv_path, index=False)

print("done")
print(df_results)
print(l2_results)

8/8 [==============================] - 0s 4ms/step


/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

8/8 [==============================] - 0s 4ms/step
done
   Strategy  Accuracy  Precision    Recall  F1 Score
0  original  0.512397   0.424772  0.512397  0.417609
1       RBP  0.363636   0.382972  0.363636  0.291528
2      MRBP  0.297521   0.144462  0.297521  0.179406
3      zero  0.392562   0.353111  0.392562  0.316578
4     noise  0.140496   0.052229  0.140496  0.049958
5      blur  0.438017   0.434523  0.438017  0.373512
     L2-Norm
0   9.565550
1  10.774890
2  10.692842
3  17.888142
4   0.170554


/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gali/anaconda3/envs/LIMEtf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
